In [26]:
##open a web page using selenium##

import itertools
# installs panda files
import pandas as pd

#install parsel package (This is super pip and only loads in the applied kernal)
#%pip install parsel
#%pip install openpyxl

from openpyxl import Workbook, load_workbook
#%pip install XlsxWriter
import openpyxl as xl
#python xlsx writer
import xlsxwriter
#what is this? 
from parsel import Selector
#import webdriver
from selenium import webdriver
#what is this?
from selenium.webdriver.common.by import By
#import options to run in headless mode (no gui)
from selenium.webdriver.chrome.options import Options
# support for waiting until page is loaded
from selenium.webdriver.support.ui import WebDriverWait
# set waiting condition (usually through CSS seletor)
from selenium.webdriver.support import expected_conditions as EC
# import select
from selenium.webdriver.support import select as Select
#import service options
from selenium.webdriver.chrome.service import Service as ChromeService # changed from just Service

#pip install of manager
#%pip install webdriver-manager

#import chrome driver manager
from webdriver_manager.chrome import ChromeDriverManager
# import page load wait time 
from selenium.webdriver.support.ui import WebDriverWait
# Keys for accessing clickable content etc.
from selenium.webdriver.common.keys import Keys
#testing to import time.sleep
import time

In [27]:
# configure webdriver options
options = Options ()
#options.add_argument("--headless") # run in headless
#options.add_argument("--window-size=1920,1080") # set window size to native GUI size
#options.add_argument("start-maximized") # open full screen

# this will disable image loading
options.add_argument('--blink-settings=imagesEnabled=false')

# set options
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)
#open page
driver.get('https://www.census.gov/popclock/world/af')
# wait for page to load
#element = WebDriverWait(driver=driver, timeout=10).until(
#    EC.presence_of_element_located((By.CSS_SELECTOR, '#content-wrapper'))
#)

element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
    (By.CSS_SELECTOR, "#country-header > h1")))

#print(driver.page_source)

In [28]:
# get country
country= []
country = [driver.find_element(By.CSS_SELECTOR,"#country-header > h1").text]

# identify page source
sel = Selector(text=driver.page_source)
# initialize lists
a = []
b =[]
c=[]
d=[]

# loop through basic facts body and gather necessary data
for item in sel.xpath("/html/body/div[2]/div[3]/div[1]" ):
       a = item.css('#basic-facts > div:nth-child(2) > h2::text').get(),
       b = item.css('#basic-facts > div:nth-child(3) > h2::text').get(),
       c = item.css('#basic-facts > div:nth-child(4) > h2::text').get(),
       d = item.css('#basic-facts > div:nth-child(5) > h2::text').get()

#create data frame
df = pd.DataFrame({'country':country,
                   'population':a,
                   'people_sqmi':b,
                   'males_per_100females':c,
                   'children_per_women':d})

# create excel
df.to_excel('UCB.xlsx', na_rep='n/a', index=False)


In [29]:
check_element = True
country_count = 0
i=2;
while i < 260:
    
    # United States transfers to a different page style at pg 245
    # United States will have to be added manually to match df
    # check for United States
    if i == 245:
        i = i + 1
    # if non data child exist count as missing country
    else:
        country_count += 1
        
    
    ## set condition for while loop##
    
    # set country number to i
    country_number = i
    # set country check variable to check if data is available
    country_check = f'/html/body/div[2]/div[1]/div/div[2]/div/ul[{country_number}]/li[2]'
    
    # find select box and left click to open country check
    driver.find_element(by=By.CSS_SELECTOR, value=
                        '#country-select > div > div.selectize-input.items.not-full.has-options > input[type=text]').click()
    
    # check element for non data child
    check_element = driver.find_elements(By.XPATH,country_check)
    
    ## exit check for condition##
    
    # Check if data exists
    while check_element != []:
        # set country number to i
        country_number = i
        # set country check variable to check if data is available
        country_check = f'/html/body/div[2]/div[1]/div/div[2]/div/ul[{country_number}]/li[2]'
    
        # find select box and left click to open country check
        driver.find_element(by=By.CSS_SELECTOR, value=
                            '#country-select > div > div.selectize-input.items.not-full.has-options > input[type=text]').click()
    
        # check element for non data child
        check_element = driver.find_elements(By.XPATH,country_check)
        
        # if non data child exists increment to next i country
        if check_element!=[] or i == 245:
            i = i + 1
        # if non data child exist count as missing country
        else:
            country_count += 1
    
    #set variable for page
    count = i
    # set selector variable for country page
    select_country = f'#country-select > div > div.selectize-dropdown.single.map-search.selectized > div > ul:nth-child({count})'

    # wait for selectable content
    element = WebDriverWait(driver=driver, timeout=10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR,
                                        '#content-wrapper')))
                                         ##country-select > div > div.selectize-input.items.not-full.has-options > input[type=text]
    # find select box and left click to open country check
    driver.find_element(by=By.CSS_SELECTOR, value=
                        '#country-select > div > div.selectize-input.items.not-full.has-options > input[type=text]').click()
    
    # wait for selectable content
    element = WebDriverWait(driver=driver, timeout=10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR,
                                        '#content-wrapper')))

    # slect country and click it.
    driver.find_element(by=By.CSS_SELECTOR, value=(select_country)).click()

    # wait for page to load
    #element = WebDriverWait(driver=driver, timeout=10).until(
     #   EC.presence_of_element_located((By.CSS_SELECTOR,'#content-wrapper')))
        
    time.sleep(4)
    
    # get country
    country= []
    country = [driver.find_element(By.CSS_SELECTOR,"#country-header > h1").text]
    
    # identify page source
    sel = Selector(text=driver.page_source)
    # initialize lists
    a = []
    b =[]
    c=[]
    d=[]
    # loop through basic facts body and gather necessary data
    for item in sel.xpath("/html/body/div[2]/div[3]/div[1]" ):
        
        #element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
         #   (By.CSS_SELECTOR, "#basic-facts > div:nth-child(2) > h2"))),
        a = item.css('#basic-facts > div:nth-child(2) > h2::text').get(),
        
        #element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
         #   (By.CSS_SELECTOR, "#basic-facts > div:nth-child(3) > h2"))),
        b = item.css('#basic-facts > div:nth-child(3) > h2::text').get(),
        
        #element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
         #   (By.CSS_SELECTOR, "#basic-facts > div:nth-child(4) > h2"))),
        c = item.css('#basic-facts > div:nth-child(4) > h2::text').get(),
        
        #element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
         #   (By.CSS_SELECTOR, "#basic-facts > div:nth-child(5) > h2"))),
        d = item.css('#basic-facts > div:nth-child(5) > h2::text').get()

    #create data frame
    df = pd.DataFrame({'country':country,
                       'population':a,
                       'people_sqmi':b,
                       'males_per_100females':c,
                       'children_per_women':d})
    
    # append excel file with new data loop
    rows = df.values.tolist()
    workbook = load_workbook(filename='UCB.xlsx')
    sheet = workbook['Sheet1']
    for row in rows:
        sheet.append(row)
    workbook.save(filename='UCB.xlsx')
    
    #increment count
    i = i + 1

In [30]:
driver.quit()

In [31]:
i

260